<a href="https://colab.research.google.com/github/rajkorde/aispikes/blob/main/training/quant_lora/QuantizeFintune_Llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate peft bitsandbytes datasets evaluate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip freeze

## Load and quantize model with bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [ ]:
model_name = "adept/persimmon-8b-base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
)

model_for_lora = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
)

In [ ]:
list(model_for_lora.parameters())

In [ ]:
print(f"Allocated VRAM: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Reserved VRAM : {torch.cuda.memory_reserved() / 1e9:.2f} GB")

In [ ]:
total_params = sum(p.numel() for p in model_for_lora.parameters())
total_size_bytes = sum(p.numel() * p.element_size() for p in model_for_lora.parameters())
print(f"Total parameters: {total_params:,}")
print(f"Model size: {total_size_bytes / (1024 ** 3):.2f} GB")

In [ ]:
type(model_for_lora)

In [ ]:
from bitsandbytes.nn import Linear4bit

any_4bit = any(isinstance(m, Linear4bit) for m in model_for_lora.modules())
print(f"Uses 4-bit layers: {any_4bit}")

## Finetune with LoRA

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset

In [ ]:
lora_config = LoraConfig(r=8, lora_alpha=16, target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"], lora_dropout=0.1, bias="none", task_type=TaskType.CAUSAL_LM)

In [ ]:
model = get_peft_model(model_for_lora, lora_config)

In [ ]:
for name, module in model.named_modules():
    if "Linear" in str(type(module)):
        print(name)


In [ ]:
dataset = load_dataset("tatsu-lab/alpaca")["train"]

In [ ]:
dataset

In [ ]:
dataset[2]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.unk_token

In [ ]:
def format_batch(batch):
    texts = [
        f"{inst.strip()}\n{inp.strip()}\n{out.strip()}" if inp.strip()
        else f"{inst.strip()}\n{out.strip()}"
        for inst, inp, out in zip(batch["instruction"], batch["input"], batch["output"])
    ]
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized["labels"] = [
        [(tid if tid != tokenizer.pad_token_id else -100) for tid in ids]
        for ids in tokenized["input_ids"]
    ]
    return tokenized


In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
%env CUDA_LAUNCH_BLOCKING=1

In [ ]:
tokenized = dataset.map(format_batch, batched=True)

In [ ]:
tokenized

In [ ]:
print(tokenizer.pad_token_id)
print(tokenized[0]["input_ids"][:10])

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    num_train_epochs=1,
    output_dir="./lora-output",
    logging_steps=10,
    fp16=True,
)

In [ ]:
# vocab_size = tokenizer.vocab_size

# def check_for_bad_labels(dataset):
#     for i, row in enumerate(dataset):
#         for token_id in row["labels"]:
#             if token_id >= vocab_size or token_id < -100:
#                 print(f"Bad token at row {i}: {token_id}")
#                 return
#     print("All labels look good.")

# check_for_bad_labels(tokenized)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
trainer.train()

## Evaluations

In [ ]:
import evaluate
from datasets import load_dataset

In [ ]:
dataset = load_dataset("gsm8k", "main")["test"].select(range(100))

In [ ]:
dataset

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
def run_eval(model, tokenizer, dataset):
  correct = 0
  for sample in dataset:
    input_text = "Q: " + sample["question"] + "\nA: "
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=64)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if sample["answer"].strip() in answer:
      correct += 1
  return correct / len(dataset)